In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [116]:
tf_data = tf.data.experimental.make_csv_dataset('./sales_train_validation.csv',
 batch_size=32,
 column_defaults=[*['']*6, *[0.]*1913],
 num_epochs=9)

In [31]:
ts_column_names = [f'd_{i}' for i in range(1, 1914)]
ts_column_names[:5]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5']

In [98]:
def build_tf_dataset(batch, ts_column_names):
    item_id = batch['item_id']
    dept_id = batch['dept_id']
    cat_id = batch['cat_id']
    store_id = batch['store_id']
    state_id = batch['state_id']

    ## ts data
    ts_data = [v for k, v in batch.items() if k in ts_column_names]

    ts_data = tf.transpose(ts_data)
    ts_train = ts_data[:, :1885, np.newaxis]
    

    ts_test = []
    lookAhead = 28
    steps = 1885
    # print(batch.shape)
    for step in range(steps):
        ts_test.append(ts_data[:, step+1:step+1+lookAhead])

    Y_labels = tf.stack(ts_test, axis=1)




    return ((item_id, dept_id, cat_id, store_id, state_id, ts_train), (Y_labels))

In [99]:
for i in tf_data.take(2).map(lambda batch: build_tf_dataset(batch, ts_column_names)):
    print(i)

((<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'HOBBIES_2_050', b'FOODS_2_174', b'HOBBIES_2_099', b'FOODS_1_019',
       b'HOBBIES_1_024', b'HOBBIES_1_268', b'FOODS_3_042',
       b'HOUSEHOLD_2_445', b'FOODS_2_064', b'HOUSEHOLD_1_193',
       b'HOBBIES_1_078', b'FOODS_3_307', b'HOUSEHOLD_1_136',
       b'HOUSEHOLD_1_145', b'FOODS_1_125', b'HOUSEHOLD_1_142',
       b'FOODS_3_178', b'HOUSEHOLD_2_082', b'FOODS_3_134',
       b'HOUSEHOLD_1_019', b'FOODS_2_186', b'FOODS_1_091',
       b'HOBBIES_1_263', b'HOUSEHOLD_2_484', b'FOODS_2_127',
       b'HOUSEHOLD_2_045', b'FOODS_2_015', b'HOBBIES_1_069',
       b'FOODS_2_224', b'HOBBIES_2_027', b'HOBBIES_2_030',
       b'HOUSEHOLD_2_227'], dtype=object)>, <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'HOBBIES_2', b'FOODS_2', b'HOBBIES_2', b'FOODS_1', b'HOBBIES_1',
       b'HOBBIES_1', b'FOODS_3', b'HOUSEHOLD_2', b'FOODS_2',
       b'HOUSEHOLD_1', b'HOBBIES_1', b'FOODS_3', b'HOUSEHOLD_1',
       b'HOUSEHOLD_1', b'FOODS_1', b'HOUSEH

# Load Vocabs

In [77]:
#load vocabs
vocab_item_id = np.load('vocab_item_id.npy', allow_pickle=True)
vocab_dept_id = np.load('vocab_dept_id.npy', allow_pickle=True)
vocab_cat_id = np.load('vocab_cat_id.npy', allow_pickle=True)
vocab_store_id = np.load('vocab_store_id.npy', allow_pickle=True)
vocab_state_id = np.load('vocab_state_id.npy', allow_pickle=True)

In [82]:
help(tf.keras.layers.Input)

Help on function Input in module keras.engine.input_layer:

Input(shape=None, batch_size=None, name=None, dtype=None, sparse=None, tensor=None, ragged=None, type_spec=None, **kwargs)
    `Input()` is used to instantiate a Keras tensor.
    
    A Keras tensor is a symbolic tensor-like object,
    which we augment with certain attributes that allow us to build a Keras model
    just by knowing the inputs and outputs of the model.
    
    For instance, if `a`, `b` and `c` are Keras tensors,
    it becomes possible to do:
    `model = Model(input=[a, b], output=c)`
    
    Args:
        shape: A shape tuple (integers), not including the batch size.
            For instance, `shape=(32,)` indicates that the expected input
            will be batches of 32-dimensional vectors. Elements of this tuple
            can be None; 'None' elements represent dimensions where the shape is
            not known.
        batch_size: optional static batch size (integer).
        name: An optional name

In [211]:
def repeat_and_concat_layer(ts_layer, *layers_to_concatenate):

    repeated_layers = []

    for layer in layers_to_concatenate:
        repeated_layers.append(tf.keras.backend.tile(tf.keras.backend.expand_dims(layer, 1), [1, tf.keras.backend.shape(ts_layer)[1],1]))

    repeated_layers.append(ts_layer)

    return tf.keras.layers.concatenate(repeated_layers, axis=-1)



In [108]:
# Simple ts model
input_item = tf.keras.layers.Input((), dtype=tf.string, name='item_input')
input_dept = tf.keras.layers.Input((), dtype=tf.string, name='dept_input')
input_cat = tf.keras.layers.Input((), dtype=tf.string, name='cat_input')
input_store = tf.keras.layers.Input((), dtype=tf.string, name='store_input')
input_state = tf.keras.layers.Input((), dtype=tf.string, name='state_input')

#ts_input
input_ts = tf.keras.layers.Input((None,1), dtype=tf.float32, name='ts_input')

##preprocess item_id inputs
lookup_item = tf.keras.layers.StringLookup(output_mode='int', vocabulary=vocab_item_id)(input_item)
emb_item = tf.keras.layers.Embedding(input_dim=3050, output_dim=32)(lookup_item)
rep_emb_item = tf.keras.layers.RepeatVector((1885))(emb_item)


# dept_input
lookup_dept = tf.keras.layers.StringLookup(output_mode='one_hot', vocabulary=vocab_dept_id)(input_dept)
rep_lookup_dept = tf.keras.layers.RepeatVector((1885))(lookup_dept)

# cat_input
lookup_cat = tf.keras.layers.StringLookup(output_mode='one_hot', vocabulary=vocab_cat_id)(input_cat)
rep_lookup_cat = tf.keras.layers.RepeatVector((1885))(lookup_cat)

# store_input
lookup_store = tf.keras.layers.StringLookup(output_mode='one_hot', vocabulary=vocab_store_id)(input_store)
rep_lookup_store = tf.keras.layers.RepeatVector((1885))(lookup_store)


# state_input
lookup_state = tf.keras.layers.StringLookup(output_mode='one_hot', vocabulary=vocab_state_id)(input_state)
rep_lookup_state = tf.keras.layers.RepeatVector((1885))(lookup_state)




## Concat layer

concat_layer = tf.keras.layers.concatenate([input_ts,
rep_emb_item,
 rep_lookup_dept,
 rep_lookup_cat,
 rep_lookup_store,
 rep_lookup_state], axis=-1)


## Simple_model
layer1_conv1D = tf.keras.layers.Conv1D(filters=10, kernel_size=2, padding='causal', dilation_rate=2, name='conv_layer1', activation='selu')(concat_layer)

layer2_conv1D = tf.keras.layers.Conv1D(filters=10, kernel_size=2, padding='causal', dilation_rate=4, name='conv_layer2', activation='selu')(layer1_conv1D)

layer3_conv1D = tf.keras.layers.Conv1D(filters=10, kernel_size=2, padding='causal', dilation_rate=6, name='conv_layer3', activation='selu')(layer2_conv1D)

layer4_conv1D = tf.keras.layers.Conv1D(filters=10, kernel_size=2, padding='causal', dilation_rate=8, name='conv_layer4', activation='selu')(layer3_conv1D)


## output dense

output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=28, activation='relu'))(layer4_conv1D)


simple_ts_model = tf.keras.models.Model(inputs=(input_item, input_dept, input_cat, input_store, input_state, input_ts), outputs=(output))




In [109]:
lookup_state(['TX'])

TypeError: 'KerasTensor' object is not callable

In [110]:
simple_ts_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 item_input (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 string_lookup_25 (StringLookup  (None,)             0           ['item_input[0][0]']             
 )                                                                                                
                                                                                                  
 dept_input (InputLayer)        [(None,)]            0           []                               
                                                                                                  
 cat_input (InputLayer)         [(None,)]            0           []                         

In [91]:
tf.keras.utils.plot_model(simple_ts_model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [86]:
import os
os.listdir()

['.vscode',
 'calendar.csv',
 'data_exploration.ipynb',
 'sales_train_evaluation.csv',
 'sales_train_validation.csv',
 'sample_submission.csv',
 'sell_prices.csv',
 'simple_ts_model.ipynb',
 'vocab_cat_id.npy',
 'vocab_dept_id.npy',
 'vocab_item_id.npy',
 'vocab_state_id.npy',
 'vocab_store_id.npy']

In [113]:
simple_ts_model.compile(optimizer='nadam', loss='mae')

In [117]:
simple_ts_model.fit(tf_data.shuffle(100).map(lambda batch: build_tf_dataset(batch=batch, ts_column_names=ts_column_names)))

8576/8576 [==============================] - 2381s 275ms/step - loss: 0.7868


In [118]:
simple_ts_model.save('./simple_ts_model')

INFO:tensorflow:Assets written to: ./simple_ts_model\assets


In [141]:
tf_pred_data = tf.data.experimental.make_csv_dataset('./sales_train_validation.csv',
 batch_size=32,
 column_defaults=[*['']*6, *[0.]*1913],
 num_epochs=1, 
 shuffle=False)

In [149]:
predictions=simple_ts_model.predict(tf_pred_data.map(lambda batch: build_tf_dataset(batch=batch, ts_column_names=ts_column_names)))

In [150]:
relevant_predictions = tf.math.ceil(predictions[:, -1])

In [158]:
df_val_id = pd.read_csv('./sales_train_validation.csv')['id']

In [159]:
df_val_pred = pd.DataFrame(relevant_predictions)

In [162]:
df_val_pred_complete = pd.concat([df_val_id, df_val_pred], axis=1)

In [163]:
df_val_pred_complete

,id,0,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,24,25,26,27
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,HOBBIES_1_005_CA_1_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30486,FOODS_3_824_WI_3_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30487,FOODS_3_825_WI_3_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30488,FOODS_3_826_WI_3_validation,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,...,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0


In [168]:
df_val_pred_complete.to_csv('./predicted_validate_df.csv', index=False)

In [196]:
tf_eval_data = tf.data.experimental.make_csv_dataset('./sales_train_evaluation.csv',
 batch_size=32,
 column_defaults=[*['']*6, *[0.]*1941],
 num_epochs=1, 
 shuffle=False)

In [197]:
def build_tf_eval_dataset(batch, ts_column_names):
    item_id = batch['item_id']
    dept_id = batch['dept_id']
    cat_id = batch['cat_id']
    store_id = batch['store_id']
    state_id = batch['state_id']

    ## ts data
    ts_data = [v for k, v in batch.items() if k in ts_column_names]

    ts_data = tf.transpose(ts_data)
    ts_train = ts_data[:, 28:1941, np.newaxis]
    

    




    return ((item_id, dept_id, cat_id, store_id, state_id, ts_train), ())

In [198]:
eval_predictions=simple_ts_model.predict(tf_eval_data.map(lambda batch: build_tf_eval_dataset(batch=batch, ts_column_names=ts_column_names)))

In [199]:
relevant_eval_predictions = tf.math.ceil(eval_predictions[:, -1])

In [200]:
df_eval_id = pd.read_csv('./sales_train_evaluation.csv')['id']

In [201]:
df_eval_pred = pd.DataFrame(relevant_eval_predictions)

In [202]:
df_eval_pred_complete = pd.concat([df_eval_id, df_eval_pred], axis=1)

In [203]:
df_eval_pred_complete

,id,0,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,24,25,26,27
0,HOBBIES_1_001_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_evaluation,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,HOBBIES_1_005_CA_1_evaluation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30486,FOODS_3_824_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30487,FOODS_3_825_WI_3_evaluation,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30488,FOODS_3_826_WI_3_evaluation,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [204]:
complete_preds = pd.concat([df_val_pred_complete, df_eval_pred_complete], axis=0)

In [205]:
complete_preds

,id,0,1,2,3,4,5,6,7,8,...,18,19,20,21,22,23,24,25,26,27
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,HOBBIES_1_005_CA_1_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30486,FOODS_3_824_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30487,FOODS_3_825_WI_3_evaluation,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30488,FOODS_3_826_WI_3_evaluation,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [208]:
complete_preds.columns = ['id', *[f'F{i}' for i in range(1,29)]]

In [209]:
complete_preds

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,HOBBIES_1_005_CA_1_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30486,FOODS_3_824_WI_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30487,FOODS_3_825_WI_3_evaluation,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30488,FOODS_3_826_WI_3_evaluation,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [210]:
complete_preds.to_csv('./complete_preds_simple_ts.csv', index=False)